In [10]:
import numpy as np
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalizationV2

In [11]:
train_path = '../data/black_box_single/seg_train/'
test_path = '../../normal/archive/seg_test/'

In [12]:
train_datagen = ImageDataGenerator(
                    rescale = 1/255,
                    shear_range = 0.2,
                    zoom_range = 0.2,
                    horizontal_flip = True,
                    vertical_flip = False)
training_set = train_datagen.flow_from_directory(train_path,
                                                 target_size = (64, 64),
                                                 class_mode = 'categorical')

Found 14034 images belonging to 6 classes.


In [13]:
test_datagen = ImageDataGenerator(rescale = 1/255)
test_set = test_datagen.flow_from_directory(test_path,
                                            target_size = (64, 64),
                                            class_mode = 'categorical')

Found 3000 images belonging to 6 classes.


In [14]:
cnn = Sequential()
cnn

In [15]:
cnn.add(Convolution2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(BatchNormalizationV2())
cnn.add(MaxPooling2D(pool_size=(2,2)))

cnn.add(Convolution2D(filters=64, kernel_size=3, activation='relu'))
cnn.add(BatchNormalizationV2())
cnn.add(MaxPooling2D(pool_size=(2,2)))

cnn.add(Convolution2D(filters=128, kernel_size=3, activation='relu'))
cnn.add(BatchNormalizationV2())
cnn.add(MaxPooling2D(pool_size=(2,2)))

cnn.add(Convolution2D(filters=256, kernel_size=3, activation='relu'))
cnn.add(BatchNormalizationV2())
cnn.add(MaxPooling2D(pool_size=(2,2)))

cnn.add(Dropout(0.4)),

cnn.add(Flatten())

cnn.add(Dense(units=256, activation='relu'))
cnn.add(BatchNormalizationV2())
cnn.add(Dropout(0.25))

cnn.add(Dense(units=128, activation='relu'))
cnn.add(BatchNormalizationV2())
cnn.add(Dropout(0.25))

cnn.add(Dense(units=64, activation='relu'))
cnn.add(BatchNormalizationV2())
cnn.add(Dropout(0.25))

cnn.add(Dense(units=24, activation='relu'))
cnn.add(BatchNormalizationV2())
cnn.add(Dropout(0.25))

cnn.add(Dense(units=6, activation='softmax'))

In [16]:
cnn.compile(optimizer = 'sgd', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [17]:
%%time
cnn.fit(training_set, steps_per_epoch= len(training_set), validation_data = test_set, validation_steps=len(test_set), epochs = 10)

Epoch 1/10


2022-08-30 17:16:02.082622: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


439/439 [==============================] - ETA: 0s - loss: 1.5836 - accuracy: 0.3929

2022-08-30 17:16:33.456880: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


439/439 [==============================] - 36s 75ms/step - loss: 1.5836 - accuracy: 0.3929 - val_loss: 1.3146 - val_accuracy: 0.4567
Epoch 2/10
439/439 [==============================] - 29s 67ms/step - loss: 1.2003 - accuracy: 0.5238 - val_loss: 1.0206 - val_accuracy: 0.6167
Epoch 3/10
439/439 [==============================] - 36s 81ms/step - loss: 1.0903 - accuracy: 0.5792 - val_loss: 1.0288 - val_accuracy: 0.6140
Epoch 4/10
439/439 [==============================] - 38s 86ms/step - loss: 1.0250 - accuracy: 0.6085 - val_loss: 1.0571 - val_accuracy: 0.5737
Epoch 5/10
439/439 [==============================] - 34s 78ms/step - loss: 0.9524 - accuracy: 0.6479 - val_loss: 0.8542 - val_accuracy: 0.6883
Epoch 6/10
439/439 [==============================] - 38s 87ms/step - loss: 0.8937 - accuracy: 0.6739 - val_loss: 0.8081 - val_accuracy: 0.7013
Epoch 7/10
439/439 [==============================] - 34s 77ms/step - loss: 0.8574 - accuracy: 0.6849 - val_loss: 0.8779 - val_accuracy: 0.6813
Epo

In [18]:
accuracy=cnn.evaluate(test_set, steps=len(test_set),verbose=2)
print('Accuracy of the model on the test set: ',accuracy[1])

94/94 - 2s - loss: 0.8212 - accuracy: 0.6907 - 2s/epoch - 20ms/step
Accuracy of the model on the test set:  0.690666675567627
